In [101]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn import metrics
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from numpy import array
from keras import optimizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [102]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.io as pio
pio.renderers.default = "vscode"


plt.style.use('default')
%matplotlib inline

In [103]:
dataframe = pd.read_csv('clean_data.csv', usecols=[5], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')

In [104]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(dataset)

MinMaxScaler()

In [105]:
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

	#dataX -> 100 200 300 Y -> 400 
	#dataY -> 200 300 400 Y -> 500 

# SVR

In [106]:
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [107]:
#for plot
look_back = 3
data = pd.read_csv('clean_data.csv', index_col=0)
data = data.iloc[train_size+look_back+1:,:]
data.sort_index(ascending=True, inplace=True)

In [108]:
train = scaler.transform(train)
test = scaler.transform(test)

In [109]:
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [110]:
trainX1 = np.reshape(trainX, (-1, 3))
trainY1 = np.reshape(trainY, (917, 1))

regressor=SVR(kernel='rbf', C=10)
regressor.fit(trainX1, trainY1)

print(trainX1.shape)
print(trainY1.shape)
 

import joblib
joblib.dump(regressor, "svr.gz")

(917, 3)
(917, 1)


C:\Users\Tejas\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



['svr.gz']

In [111]:
pred = regressor.predict(testX)
series_results = pd.DataFrame(scaler.inverse_transform([pred]).flatten(), columns=['SVR_10'])


print(f"Mean Abs Error: {metrics.mean_absolute_error(testY, pred)}")
print(f"Mean Sq Error: {metrics.mean_squared_error(testY, pred)}")
print(f"Root Mean Error: {np.sqrt(metrics.mean_squared_error(testY, pred))}")

Mean Abs Error: 0.061743450538459484
Mean Sq Error: 0.0067924560592082725
Root Mean Error: 0.08241635795889231


In [112]:
fig = go.Figure()
fig.update_layout(
    title="Comparison of predicted AQI and Measured AQI",
    xaxis_title="Date",
    yaxis_title="AQI Level",
    legend_title="Legend",
)

fig.add_trace(go.Scatter(x=series_results.index, y=scaler.inverse_transform([testY]).flatten(), name="Actual", line_shape='spline'))
fig.add_trace(go.Scatter(x=series_results.index, y=scaler.inverse_transform([pred]).flatten()   , name="SVR", line_shape='spline'))
fig.show()

### Cross VAlidation

# Polynomial Regression with degree=10

In [13]:
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [14]:
#for plot
look_back = 3
data = pd.read_csv('clean_data.csv', index_col=0)
data = data.iloc[train_size+look_back+1:,:]
data.sort_index(ascending=True, inplace=True)

In [15]:
train = scaler.transform(train)
test = scaler.transform(test)

In [16]:
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [17]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [18]:
print(trainX.shape)
print(trainY.shape)

(917, 1, 3)
(917,)


In [19]:
from sklearn.preprocessing import PolynomialFeatures  
poly_regs = PolynomialFeatures(degree= 10)  

trainX1 = np.reshape(trainX, (-1, 3))
trainY1 = np.reshape(trainY, (917, 1))
print(trainX1.shape)
print(trainY1.shape)

x_train_poly = poly_regs.fit_transform(trainX1)  
poly_lin_reg = LinearRegression()  
poly_lin_reg.fit(x_train_poly, trainY1)  

import joblib
joblib.dump(poly_lin_reg, "poly_reg.gz")

(917, 3)
(917, 1)


['poly_reg.gz']

In [20]:
testX1 = np.reshape(testX, (-1, 3))
testY1 = np.reshape(testY, (450, 1))

print(testX1.shape)
print(testY1.shape)

x_test_poly = poly_regs.fit_transform(testX1)  
pred = poly_lin_reg.predict(x_test_poly)

series_results["Poly_Reg"] = scaler.inverse_transform(pred)

series_results.index = data.index
series_results.sort_index(ascending=True, inplace=True)


print(f"Mean Abs Error: {metrics.mean_absolute_error(testY1, pred)}")
print(f"Mean Sq Error: {metrics.mean_squared_error(testY1, pred)}")
print(f"Root Mean Error: {np.sqrt(metrics.mean_squared_error(testY1, pred))}")

(450, 3)
(450, 1)
Mean Abs Error: 0.06037198752164841
Mean Sq Error: 0.009249642491340637
Root Mean Error: 0.09617505967617035


In [21]:
fig = go.Figure()
fig.update_layout(
    title="Comparison of predicted AQI and Measured AQI",
    xaxis_title="Date",
    yaxis_title="AQI Level",
    legend_title="Legend",
)

fig.add_trace(go.Scatter(x=series_results.index, y=scaler.inverse_transform(testY1).flatten(), name="Actual", line_shape='spline'))
fig.add_trace(go.Scatter(x=series_results.index, y=scaler.inverse_transform(pred).flatten()   , name="Polynomial reg", line_shape='spline'))
fig.show()

# ARIMA with parameters p=1, d=1, q=1

In [22]:
from statsmodels.tsa.arima.model import ARIMA

In [23]:
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [24]:
train = scaler.transform(train)
test = scaler.transform(test)

In [25]:
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [26]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [27]:
history = [x for x in train]

In [28]:
predictions = list()

In [29]:
predictions.clear()
for t in range(len(test)):
	model = ARIMA(history, order=(1,1,1))
	model_fit = model.fit()
	output = model_fit.forecast()
	yhat = output[0]
	predictions.append(yhat)
	obs = test[t]
	history.append(obs)
	print('predicted=%f, expected=%f' % (yhat, obs))

predicted=0.168476, expected=0.184211


C:\Users\Tejas\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



predicted=0.170054, expected=0.164474
predicted=0.166958, expected=0.177632
predicted=0.170436, expected=0.184211
predicted=0.173888, expected=0.180921
predicted=0.174993, expected=0.164474
predicted=0.171226, expected=0.171053
predicted=0.171844, expected=0.213816
predicted=0.184594, expected=0.200658
predicted=0.186555, expected=0.184211
predicted=0.184451, expected=0.164474
predicted=0.178440, expected=0.177632
predicted=0.179579, expected=0.184211
predicted=0.181171, expected=0.180921
predicted=0.180794, expected=0.243421
predicted=0.199689, expected=0.263158
predicted=0.214548, expected=0.351974
predicted=0.251428, expected=0.276316
predicted=0.248978, expected=0.361842
predicted=0.280745, expected=0.338816
predicted=0.290524, expected=0.351974
predicted=0.304646, expected=0.335526
predicted=0.309496, expected=0.342105
predicted=0.316974, expected=0.338816
predicted=0.321225, expected=0.266447
predicted=0.302691, expected=0.319079
predicted=0.311269, expected=0.335526
predicted=0.

C:\Users\Tejas\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



predicted=0.112498, expected=0.115132
predicted=0.113062, expected=0.111842
predicted=0.112483, expected=0.134868
predicted=0.119504, expected=0.128289
predicted=0.120748, expected=0.131579
predicted=0.123385, expected=0.118421
predicted=0.121049, expected=0.121711
predicted=0.121509, expected=0.125000
predicted=0.122575, expected=0.131579
predicted=0.125139, expected=0.121711
predicted=0.123450, expected=0.131579
predicted=0.126146, expected=0.154605
predicted=0.134468, expected=0.138158
predicted=0.133667, expected=0.148026
predicted=0.137698, expected=0.138158
predicted=0.136841, expected=0.144737
predicted=0.139169, expected=0.164474
predicted=0.146500, expected=0.157895
predicted=0.148306, expected=0.148026
predicted=0.147292, expected=0.167763
predicted=0.153588, expected=0.171053
predicted=0.157652, expected=0.157895
predicted=0.156431, expected=0.180921
predicted=0.163909, expected=0.213816
predicted=0.177797, expected=0.194079
predicted=0.179389, expected=0.180921
predicted=0.

C:\Users\Tejas\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



predicted=0.175382, expected=0.154605
predicted=0.172216, expected=0.075658
predicted=0.143637, expected=0.088816
predicted=0.133071, expected=0.154605
predicted=0.144223, expected=0.213816
predicted=0.165128, expected=0.055921
predicted=0.126040, expected=0.049342
predicted=0.108804, expected=0.032895
predicted=0.090717, expected=0.016447
predicted=0.072907, expected=0.213816
predicted=0.123016, expected=0.072368
predicted=0.098104, expected=0.062500
predicted=0.089443, expected=0.029605
predicted=0.073277, expected=0.020724
predicted=0.061008, expected=0.154605
predicted=0.094378, expected=0.059211
predicted=0.077454, expected=0.069079
predicted=0.076602, expected=0.154605
predicted=0.101777, expected=0.078947
predicted=0.089485, expected=0.105263
predicted=0.095448, expected=0.013158
predicted=0.068744, expected=0.121711
predicted=0.090690, expected=0.108553
predicted=0.093279, expected=0.098684
predicted=0.093492, expected=0.134868
predicted=0.105926, expected=0.072368
predicted=0.

C:\Users\Tejas\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



predicted=0.115488, expected=0.032895
predicted=0.090505, expected=0.055921
predicted=0.085045, expected=0.009868
predicted=0.063609, expected=0.171053
predicted=0.103666, expected=0.197368
predicted=0.127314, expected=0.266447
predicted=0.165400, expected=0.217105
predicted=0.171883, expected=0.062500
predicted=0.131549, expected=0.000000
predicted=0.095738, expected=0.171053
predicted=0.130381, expected=0.105263
predicted=0.117951, expected=0.164474
predicted=0.134390, expected=0.151316
predicted=0.136739, expected=0.101974
predicted=0.123919, expected=0.134868
predicted=0.129774, expected=0.095395
predicted=0.118076, expected=0.243421
predicted=0.161107, expected=0.138158
predicted=0.145089, expected=0.042763
predicted=0.112608, expected=0.253289
predicted=0.165340, expected=0.203947
predicted=0.168788, expected=0.177632
predicted=0.168017, expected=0.184211
predicted=0.172270, expected=0.138158
predicted=0.159993, expected=0.190789
predicted=0.172212, expected=0.263158
predicted=0.

In [30]:
normal = [[val] for val in predictions]
testPredict = scaler.inverse_transform(normal[:-4])
testY2 = scaler.inverse_transform([testY])

In [31]:
print(len(testY2[0]))
print(len(testPredict))

450
450


In [32]:
print(f"Mean Abs Error: {metrics.mean_absolute_error(testY2[0], testPredict[:,0])}")
print(f"Mean Sq Error: {metrics.mean_squared_error(testY2[0], testPredict[:,0])}")
print(f"Root Mean Error: {np.sqrt(metrics.mean_squared_error(testY2[0], testPredict[:,0]))}")

Mean Abs Error: 17.81890581794742
Mean Sq Error: 619.9335922393743
Root Mean Error: 24.898465660344904


In [33]:
series_results["ARIMA"] = testPredict
fig = go.Figure()
fig.update_layout(
    title="Comparison of predicted AQI and Measured AQI",
    xaxis_title="Date",
    yaxis_title="AQI Level",
    legend_title="Legend",
)

fig.add_trace(go.Scatter(x=series_results.index, y=scaler.inverse_transform(testY1).flatten(), name="Actual", line_shape='spline'))
fig.add_trace(go.Scatter(x=series_results.index, y=testPredict.flatten() , name="ARIMA", line_shape='spline'))
fig.show()

### Cross Validation

In [99]:
import numpy as np
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

df = pd.read_csv('clean_data.csv', usecols=[5])
df.dropna(inplace=True)

tscv = TimeSeriesSplit(n_splits = 5)
rmse = []
for train_index, test_index in tscv.split(df):
    cv_train, cv_test = df.iloc[train_index], df.iloc[test_index]
    model = ARIMA(cv_train.AQI, order=(1, 1, 1)).fit()
    predictions = model.predict(cv_test.index.values[0], cv_test.index.values[-1])
    true_values = cv_test.AQI
    rmse.append(np.sqrt(mean_squared_error(true_values, predictions)))
print(rmse)
print('Avg. RMSE= ', np.mean(rmse))

[62.232628045645015, 54.2487716314501, 39.13455564978794, 29.641370363049198, 34.852749548981905]
Avg. RMSE=  44.02201504778283


# LSTM (Look_back=3)

In [114]:
import tensorflow as tf
tf.random.set_seed(7)

In [115]:
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [116]:
train = scaler.transform(train)
test = scaler.transform(test)

In [117]:
look_back = 3
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [118]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [119]:
from tensorflow import keras
model = keras.models.load_model('AQI_LSTM.h5')

In [120]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

15/15 [==============================] - 0s 14ms/step


In [121]:
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY1 = scaler.inverse_transform([testY])

In [122]:
series_results["LSTM_3"] = testPredict
print(f"Mean Abs Error: {metrics.mean_absolute_error(testY1[0], testPredict[:,0])}")
print(f"Mean Sq Error: {metrics.mean_squared_error(testY1[0], testPredict[:,0])}")
print(f"Root Mean Error: {np.sqrt(metrics.mean_squared_error(testY1[0], testPredict[:,0]))}")

Mean Abs Error: 14.568286552699073
Mean Sq Error: 420.34122893641717
Root Mean Error: 20.502224975265907


In [123]:
fig = go.Figure()
fig.update_layout(
    title="Comparison of predicted AQI and Measured AQI",
    xaxis_title="Date",
    yaxis_title="AQI Level",
    legend_title="Legend",
)

fig.add_trace(go.Scatter(x=series_results.index, y=testY1.flatten(), name="Actual", line_shape='spline'))
fig.add_trace(go.Scatter(x=series_results.index, y=testPredict.flatten()  , name="LSTM", line_shape='spline'))
fig.show()

### Cross Validation

# LSTM (Look_back=10)

In [125]:
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [126]:
train = scaler.transform(train)
test = scaler.transform(test)

In [127]:
look_back = 10
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [128]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [129]:
from tensorflow import keras
model = keras.models.load_model('AQI_LSTM_new.h5')

In [130]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

14/14 [==============================] - 0s 15ms/step


In [131]:
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY1 = scaler.inverse_transform([testY])

In [132]:
series_results["LSTM_10"] = np.append(np.zeros(look_back-3), values=testPredict)
print(f"Mean Abs Error: {metrics.mean_absolute_error(testY1[0], testPredict[:,0])}")
print(f"Mean Sq Error: {metrics.mean_squared_error(testY1[0], testPredict[:,0])}")
print(f"Root Mean Error: {np.sqrt(metrics.mean_squared_error(testY1[0], testPredict[:,0]))}")

Mean Abs Error: 14.613421876821032
Mean Sq Error: 424.9508267022317
Root Mean Error: 20.61433546593806


In [133]:
fig = go.Figure()
fig.update_layout(
    title="Comparison of predicted AQI and Measured AQI",
    xaxis_title="Date",
    yaxis_title="AQI Level",
    legend_title="Legend",
)

fig.add_trace(go.Scatter(x=series_results.index, y=testY1.flatten(), name="Actual", line_shape='spline'))
fig.add_trace(go.Scatter(x=series_results.index, y=testPredict.flatten()  , name="LSTM_10", line_shape='spline'))
fig.show()

### Cross Validation


# Comparision of various Models

In [54]:
fig = go.Figure()
fig.update_layout(
    title="Comparison of predicted AQI and Measured AQI",
    xaxis_title="Date",
    yaxis_title="AQI Level",
    legend_title="Legend",
)
fig.update_xaxes(range=['2016-10-01', '2022-03-01'])

fig.add_trace(go.Scatter(x=series_results.index, y=testY1.flatten(), name="Actual", line_shape='spline'))
fig.add_trace(go.Scatter(x=series_results.index, y=series_results.SVR_10  , name="SVR(c=10)", line_shape='spline'))
fig.add_trace(go.Scatter(x=series_results.index, y=series_results.LSTM_3  , name="LSTM(3)", line_shape='spline'))
fig.add_trace(go.Scatter(x=series_results.index, y=series_results.LSTM_10  , name="LSTM(10)", line_shape='spline'))
fig.add_trace(go.Scatter(x=series_results.index, y=series_results.Poly_Reg  , name="Polynomial(10)", line_shape='spline'))
fig.add_trace(go.Scatter(x=series_results.index, y=series_results.ARIMA  , name="ARIMA", line_shape='spline'))


fig.show()

In [ ]:
# Cross Valida